In [ ]:
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection
library(data.table)

# Ganancia total considerando los primeros 11.000 clientes
N_top <- 11000L

# Setear directorios
setwd("/content/buckets/b1/exp")


In [ ]:
library(data.table)

#-------------------------------
# 1) Lista de archivos a ensamblar
#-------------------------------


files <- c(
  "ESBL202109_B2/prediccion.txt",
  "ESBL202109_01/prediccion.txt",
  "ESBL202109_02/prediccion.txt",
  "ESBL202109_03/prediccion.txt",
  "ESBL202109_05/prediccion.txt",
  "ESBL202109_06/prediccion.txt",
)


#-------------------------------
# 2) Leer y apilar todos los archivos
#-------------------------------

dt_list <- lapply(files, function(f) {
  fread(f, sep = "\t", header = TRUE)  # usar sep="," si son CSV
})

# Verificación opcional: todas las tablas tienen las columnas esperadas
# lapply(dt_list, names)

dt_all <- rbindlist(dt_list, use.names = TRUE, fill = TRUE)

# Esperamos columnas: numero_de_cliente, foto_mes, prob

#-------------------------------
# 3) Ensamblar: promedio de prob por cliente + mes
#-------------------------------

dt_ensamble <- dt_all[
  ,
  .(prob = mean(prob, na.rm = TRUE)),
  by = .(numero_de_cliente, foto_mes)
]

# Ahora dt_ensamble tiene:
# numero_de_cliente, foto_mes, prob (promedio de los 5 modelos)

#-------------------------------
# 4) Filtrar foto_mes = 202109,
#    ordenar por prob desc,
#    tomar primeros 11000
#-------------------------------

# Mejor ser paranoico antes que descuidado
dt_202109 <- dt_ensamble[foto_mes == 202109]

# Ordenar por probabilidad en orden decreciente
setorder(dt_202109, -prob)

N_top <- min(11000L, nrow(dt_202109))  # por si hubiera menos de 11000

top_clientes <- dt_202109[seq_len(N_top), .(numero_de_cliente)]

#-------------------------------
# 5) Guardar CSV sin encabezado,
#    una sola columna: numero_de_cliente
#-------------------------------

fwrite(
  top_clientes,
  file = "submit_competencia3_mvolta.csv",
  sep = ",",
  col.names = FALSE
)
